In [15]:
%pylab inline
from __future__ import print_function
import librosa as lr
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [127]:
audio_folder = '../../audio/wav/'
feature_folder = '../../features/'
audio_list_file = '../../database/audio_files.csv'
audio_ext = '.wav'

audio_list = pd.read_csv(audio_list_file, header=-1).values
audio_list = list(audio_list.T[0])
n_audio_files = len(audio_list)
print(n_audio_files)

143


### FEATURE EXTRACTION TEST

In [144]:
hop_length = 512
file_name = audio_list[0]

y, sr = lr.load(audio_folder + file_name + audio_ext)

tempo, beat_frames = lr.beat.beat_track(y=y, hop_length=hop_length, sr=sr)
beat_times = lr.frames_to_time(beat_frames, sr=sr)
np.savetxt(file_name + '_beat_frames.csv', beat_frames, delimiter=',')
np.savetxt(file_name + '_beat_times.csv', beat_times, delimiter=',')

mfcc_frames = lr.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
mfcc_beats = lr.feature.sync(mfcc_frames, beat_frames, aggregate=np.median)
mfcc_beats_data = pd.DataFrame(np.hstack([beat_times.reshape([-1,1]), mfcc_beats.T[1:]]))
np.savetxt(file_name + '_mfcc_beat.csv', mfcc_beats_data, delimiter=',')

### FEATURE EXTRACTION LOOP

In [ ]:
hop_length = 512
for file_name in audio_list:
    print('processing ' + file_name + '...')
    y, sr = lr.load(audio_folder + file_name + audio_ext)
    
    tempo, beat_frames = lr.beat.beat_track(y=y, hop_length=hop_length, sr=sr)
    beat_times = lr.frames_to_time(beat_frames, sr=sr)
    np.savetxt(feature_folder + file_name + '_beat_frames.csv', beat_frames, delimiter=',')
    np.savetxt(feature_folder + file_name + '_beat_times.csv', beat_times, delimiter=',')
    
    mfcc_frames = lr.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
    mfcc_beats = lr.feature.sync(mfcc_frames, beat_frames, aggregate=np.median)
    mfcc_beats_data = pd.DataFrame(np.hstack([beat_times.reshape([-1,1]), mfcc_beats.T[1:]]))
    np.savetxt(feature_folder + file_name + '_mfcc_beat.csv', mfcc_beats_data, delimiter=',')

processing T001...
processing T002...
processing T003...
processing T004...
processing T005...
processing T006...
processing T007...
processing T008...
processing T009...
processing T010...
processing T011...
processing T012...
processing T013...
processing T014...
processing T015...
processing T016...
processing T017...
processing T018...
processing T020...
processing T021...
processing T022...
processing T023...
processing T024...
processing T025...
processing T026...
processing T027...

In [ ]:
import librosa as lr

def beat_aggregation(feature_data, file_name, time_col=False, agg_function=np.median):
    feat_frames = np.genfromtxt(feature_data).T
    if time_col:
        feat_frames = feat_frames[1:]
    beat_frames = np.genfromtxt(file_name + '_beat_frames.csv')
    feat_beats = lr.feature.sync(mfcc_frames, beat_frames, aggregate=agg_function)
    np.savetxt(feature_data.split('.')[-2] + '_beat.csv', mfcc_beats_data, delimiter=',')